In [ ]:
import os,sys,psycopg2,ipywidgets,solara;
from IPython.display import display;
from IPython.display import FileLink;
from contextlib import closing;

sys.path.append(os.path.join(os.path.expanduser('~'),'notebooks'));
import common;

dbse = os.environ['POSTGRESQL_DB'];
host = os.environ['POSTGRESQL_HOST'];
port = os.environ['POSTGRESQL_PORT'];
user = 'cipsrv_upload';
pswd = os.environ['POSTGRESQL_CIP_PASS'];
uptrg = os.path.join(os.path.expanduser('~'),'notebooks/upload');


In [ ]:
cs = "dbname=%s user=%s password=%s host=%s port=%s" % (
     dbse
    ,user
    ,pswd
    ,host
    ,port
);

try:
    conn = psycopg2.connect(cs);
    conn.set_session(autocommit=True);
except:
    raise Exception("database connection error");

print("database is ready");


In [ ]:
stem = ipywidgets.Text(
     value       = None
    ,placeholder = 'Enter indexing stem here.'
    ,description = 'Stem:'
    ,disabled    = False
    ,continuous_update=False
)
display(stem);

output = ipywidgets.Output();
display(output);

@output.capture()
def on_value_change(change):
    print('stem: ' + stem.value);
    
stem.observe(on_value_change,names='value');


In [ ]:
upstem = stem.value;
with closing(conn.cursor()) as cursor:
    for item in ['_control','_points','_lines','_areas','_sfid','_cip']:
        cursor.execute("SELECT COUNT(*) FROM cipsrv_upload." + upstem + item + " ");
        result = cursor.fetchone();
        print(upstem + item + ": " + str(result[0]));
    

In [ ]:
trgcsv = os.path.join(uptrg,upstem + '_cip.csv');
if os.path.exists(trgcsv):
    os.remove(trgcsv);

cmd = '-overwrite -f CSV ' + trgcsv + ' '                                              \
    + 'postgresql://' + user + ':' + pswd + '@' + host + ':' + port + '/' + dbse + ' ' \
    + '-sql "SELECT '                                                                  \
    + ' a.catchmentstatecode AS state '                                                \
    + ',a.nhdplusid          AS feature_id '                                           \
    + ',a.source_featureid   AS storigid '                                             \
    + ',a.catchmentareasqkm  AS area_sqkm '                                            \
    + 'FROM cipsrv_upload.' + upstem + '_cip a '                                       \
    + 'ORDER BY '                                                                      \
    + ' a.source_featureid '                                                           \
    + ',a.nhdplusid " '                                                                \
    + '-lco LINEFORMAT=CRLF ';

z = common.ogr2ogr(
     cmdstring = cmd
);
print(str(z[1]));


In [ ]:
import solara;

file_object = open(trgcsv,"rb");
z = solara.FileDownload(data=file_object,filename=os.path.basename(trgcsv));
display(z);
